In [45]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [47]:
# se carga el dataset
df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')
# se selecciona las columnas de interés
columnas_seleccionadas = ['overview', 'popularity', 'tagline', 'title']  # Reemplaza con los nombres de las columnas que deseas seleccionar
df = df[columnas_seleccionadas]
df.sample(10)

C:\Users\HP\AppData\Local\Temp\ipykernel_3596\2947862856.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')


,overview,popularity,tagline,title
13296,"In 1923, the young Kim Shun-Pei moves from Che...",1.727334,NaN,Blood and Bones
8787,"In this documentary, the director follows the ...",0.035437,NaN,Best Boy
43006,"A series of 32 short scenes, uniformly set in ...",0.03529,NaN,How to Live in the German Federal Republic
40812,City of God – 10 Years Later investigates what...,1.270588,NaN,City of God – 10 Years Later
20272,A horror film based on a real-life mass murder...,0.696757,NaN,Chicago Massacre: Richard Speck
6796,"After a car crash, a criminal psychologist (Ha...",10.939066,Because someone is dead doesn't mean they're g...,Gothika
22880,When heads of state gather at the G8 summit in...,0.202285,NaN,The Monster X Strikes Back: Attack the G8 Summit
19860,A documentary about filmmaker Budd Boetticher.,0.003786,NaN,Budd Boetticher: An American Original
14050,Biopic of the rock icon. Made for television,0.151378,NaN,Little Richard
1264,An ambitious female attorney wallows in excess...,2.273475,Dare to let yourself go where you've never bee...,Female Perversions


In [48]:
# Se eliminan los registros duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [49]:
#se imputan los nulos o nan con ''
titulo = df['title'].fillna('')
resumen = df['overview'].fillna('')
frase   = df['tagline'].fillna('')

In [50]:
tfidf = TfidfVectorizer(stop_words='english') # configuracion de TfidfVectorizery, en idioma ingles

In [52]:
# se crea la una matriz TF-IDF con las puntaciones del campo overview
tfidf_matrix = tfidf.fit_transform(titulo)

In [53]:
print(tfidf_matrix)

  (0, 19260)	0.5339435668750432
  (0, 20542)	0.8455201164920656
  (1, 10646)	1.0
  (2, 12911)	0.4385962787286779
  (2, 14434)	0.49506099693865796
  (2, 8663)	0.7500319417167898
  (3, 6866)	0.7982816099847494
  (3, 21743)	0.602284377317025
  (4, 9820)	0.5069599602460498
  (4, 2927)	0.61439304931765
  (4, 7111)	0.6045765291983168
  (5, 9141)	1.0
  (6, 17314)	1.0
  (7, 9632)	0.8034526222023605
  (7, 20410)	0.5953686957475602
  (8, 5290)	0.5383892659096897
  (8, 19437)	0.8426962669629108
  (9, 8404)	1.0
  (10, 15835)	0.8041548542327221
  (10, 886)	0.5944198603798071
  (11, 12048)	0.6667295668851593
  (11, 5265)	0.44741905880044924
  (11, 6047)	0.5960602909632952
  (12, 1784)	1.0
  (13, 14129)	1.0
  :	:
  (45365, 12669)	0.47019920205394805
  (45366, 5360)	0.6886927515673559
  (45366, 9136)	0.7250533042049973
  (45367, 13331)	1.0
  (45368, 13480)	1.0
  (45369, 17095)	0.6666725078218987
  (45369, 13024)	0.5325553344616729
  (45369, 19034)	0.5214715553612453
  (45370, 9553)	0.8374035427397905


In [55]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) # se calcula la matriz de coseno para los vectores.

In [56]:
def get_recommendations(df, column, value, cosine_similarities, limit=5):
    """ Retorna un dataframe de 5 registros recomendados según la 
        Frecuencia de término Frecuencia de documento inversa (TF-IDF) 
        y la similitud del coseno.
    
    Args:
        df (object): dataframe que contiene el texto a analizar. 
        column (string): nombre de la columna utilizada. 
        value (string): nombre del titulo a tener las recomendaciones.
        cosine_similarities (array): matriz de similitud del cosenol
        limit (int, optional): retorna la cantidad de registros a retornar recomendados. 
        
    Returns: 
        Pandas dataframe. 
    """
    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df.index, index=df[column]).drop_duplicates()

    # Get the index for the target value
    target_index = indices[value]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [57]:
recommendations = get_recommendations(df, 
                                      'title', 
                                      'Toy Story', 
                                      cosine_similarities)
recommendations.head(5)

,index,recommendation,cosine_similarity_score
0,2995,Toy Story 2,1.000000
1,15341,Toy Story 3,1.000000
2,4795,The Toy,0.845520
3,16146,The Toy,0.845520
4,21905,Toy Story of Terror!,0.817671
